# Exploring the NIFTY 50 index under different SMA regimes

Markets tend to be manic depressive, going from excessive optimism to excessive pessimism. The distribution of returns tend to be different based on the "regime." A simple way to demarcate these regimes is by using an SMA (Simple Moving Average) to bucket returns into binary groups. Here, we explore how returns are distributed in these buckets.

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices<-Indices()

In [ ]:
smaLb <- 50 #number of days
px <- indices$NseTimeSeries() %>%
    filter(NAME == 'NIFTY 50 TR') %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

numDays <- px %>%
    group_by(Y = year(TIME_STAMP)) %>%
    summarize(N = n())

numDays %>% print(n=Inf)

In [ ]:
pXts <- xts(px$CLOSE, px$TIME_STAMP)
names(pXts) <- c('INDEX')
pXts$SMA <- SMA(pXts[,1], smaLb)
pXts$D_RET <- dailyReturn(pXts$INDEX)

yrs <- unique(year(index(pXts)))

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)